# Telco Customer Churn Prediction Assignment Problem Statement - 7

#### Name: Yenduru Poojith Manideep
#### BITS ID: 2024ac05966@wilp.bits-pilani.ac.in

#### Objective: Predict customer churn for a telecommunications company using machine learning models

---
## Section 1: Import Libraries and Dataset

In [ ]:
# Importing Required Libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical operations
import matplotlib.pyplot as plt  # For plotting
import seaborn as sns  # For advanced visualization
import warnings
warnings.filterwarnings('ignore')

# Setting visualization style
sns.set(style='whitegrid', palette='muted', font_scale=1.1)

### Load the Dataset


In [ ]:
# Loading Telco Customer Churn Dataset
df = pd.read_csv('Telco-Dataset.csv')
print('Dataset Loaded. Shape of the Dataset is:', df.shape)

---
## Section 2: Data Visualization and Exploration

Exploratory Data Analysis (EDA) is a critical step in any machine learning project. It helps build intuition about the data's structure, distributions, imbalances, correlations, and potential issues. Visualization makes patterns and outliers visible, while summary statistics and info reveal hidden or non-obvious data problems.

### 2.a) Sanity Check: Preview Data
Before doing any kind of modeling, I first take a quick look at the dataset. This is like a sanity check to see if the data is actually loaded properly and whether the columns and rows make sense. By just previewing the first few rows, I can immediately spot if there are weird values, missing headers, or any formatting issues. This step saves time later because if the data itself is broken, then the whole analysis will go wrong.

In [ ]:
df.head(10)  # Show 10 rows for a richer preview

### 2.b) Describe and Check Dataset Details

After the preview, I used .info() and .describe() to understand the data set better. This gives me details like the number of rows, columns, datatypes, null values, and also some basic statistics such as mean, min, max, and standard deviation. These details help me confirm if the columns are in the right type (like numbers should not be stored as strings), and also if there are any obvious errors in ranges or values.

In [ ]:
# Dataset metadata & null scan
print('Dataset shape:', df.shape)

print('\nColumn Types and Null Scan:')
print(df.info())

### 2.c) Visual Exploration (Univariate and Bivariate Analysis)

In univariate analysis, I study each column separately. This helps me see how the values are distributed—whether a feature is normally distributed, skewed, or has too many repeated values. For categorical columns, I check the frequency counts to see if the dataset is balanced or heavily imbalanced. This step is important because later on, the model’s performance depends on how well these features are understood and preprocessed.

Observations :-

1. For the numerical data, the distribution of Tenure and Monthly Charges is okay, but the TotalCharges is positively skewed. I am handling this later
2. For the categorical data, we see a lot of class imbalance issue. I am handling this issue also later.


In [ ]:
# Churn value counts to check balance
plt.figure(figsize=(4,3))
sns.countplot(data=df, x='Churn')
plt.title('Churn Class Distribution')
plt.show()

# Distribution of numerical features
num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
fig, axs = plt.subplots(1, 3, figsize=(18, 4))
for i, col in enumerate(num_cols):
    sns.histplot(df[col], bins=25, kde=True, ax=axs[i])
    axs[i].set_title(f'Distribution of {col}')
plt.tight_layout()
plt.show()

# Distribution of categorical features
cat_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
            'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']

plt.figure(figsize=(16, 16))
for i, col in enumerate(cat_cols, 1):
    plt.subplot(4, 4, i)
    sns.countplot(x=col, data=df, color='steelblue')
    plt.tight_layout()
plt.suptitle('Categorical Feature Distributions', y=1.02, fontsize=16)
plt.show()

Bivariate analysis is about checking the relationship between two variables. Here, I usually compare the independent features with the target column (like churn vs tenure). This helps me see which features are actually related to the output. For example, if customers with shorter tenure are more likely to churn, then tenure is clearly an important feature. Doing this gives me early hints about which variables will be useful for the model.

My Observation:-

If we can see the first plot, Tenure vs Churn, we casn clearly see that the customers with shorter tenures are more likely got churned. SO Like this, for every Feature, I can get an insight of what would be the relation or correlation between each feature and Churn output class variable. So this is interesting for me.

In [ ]:
import math

# # Bivariate Analysis: Contract Type vs Churn
total_columns = num_cols + cat_cols
n_cols = 3   # number of plots per row (adjust if you want)
n_rows = math.ceil(len(total_columns) / n_cols)

plt.figure(figsize=(18, n_rows * 4))

for i, col in enumerate(total_columns, 1):
    plt.subplot(n_rows, n_cols, i)
    
    if col in num_cols:
        sns.histplot(x=col, hue='Churn', data=df, palette='Set2', multiple='stack', bins=30)
    else:
        sns.countplot(x=col, hue='Churn', data=df, palette='Set2')
    
    plt.title(f'Churn Distribution by {col}')
    plt.xlabel(col)
    plt.ylabel('Count')

plt.tight_layout()
plt.show()

### 2.d) Correlational Analysis

**Numerical attribute correlation:** The correlation matrix shows how features are related to each other. If two features are highly correlated, then they are almost carrying the same information. Including both may confuse the model or add unnecessary noise. By checking the correlation matrix, I can identify redundancy and decide if I need to drop or combine some features. It also helps me see which variables are strongly correlated with the target (if included).

The result of this analysis is a correlation coefficient, which typically ranges from -1 to +1.

+1: Indicates a perfect positive linear relationship. As one variable increases, the other variable increases proportionally.

-1: Indicates a perfect negative linear relationship. As one variable increases, the other variable decreases proportionally.

0: Indicates no linear relationship between the variables.

**Method That I have used:** I have used Pearson correlation matrix and computed on numerical features. This will be Visualized as a heatmap 

**Justification on Why should we use this:** Correlational analysis will be needed for the feature selection that will be performed in the next step.

Feature selection is the process of selecting a subset of relevant features to be used in building a machine learning model. The goal is to improve model performance, reduce complexity, and decrease training time. The correlation matrix is a primary tool for guiding this process.

**Correlation between Independent Features (Features vs. Features)**

The heatmap will reveal any pairs of independent variables if they are highly correlated with each other (e.g., a correlation coefficient of > 0.85 or < -0.85). This phenomenon is called multicollinearity.

Effect on Feature Selection: If two features are highly correlated, they are essentially providing redundant information. Keeping both in the model can cause several problems. Therefore, the common practice is to remove one of the highly correlated features.

Justification:

Model Stability and Reliability: Multicollinearity can make the model's coefficient estimates unstable and highly sensitive to minor changes in the data. For linear models (like Linear Regression or Logistic Regression), it becomes difficult to determine the individual contribution of each correlated feature to the prediction. By removing one, the model becomes more stable and the coefficients become more interpretable.

Reduces Redundancy and Complexity: The principle of Occam's Razor suggests that simpler models are better. If two features provide the same information, it is more efficient to use only one. This reduces the dimensionality of the data, which can lead to faster training times and a less complex, more generalizable model.

Avoids Overfitting: Including redundant features can sometimes lead to the model fitting to the noise in the data rather than the underlying signal, which results in poor performance on new, unseen data.

In [ ]:
#Correlational matrix (numerical)
plt.figure(figsize=(6,4))
corr = df[['tenure', 'MonthlyCharges']].corr(method='pearson')
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix (Numerical Features)')
plt.show()

Observation :- We only have 2 numerical features and Here both the features are not highly correlated with each other. So, There is no need to remove any feature. If any of them are highly correlated i.e., >0.85, then we should ideally have checked for correlation with output class and only keep one and remove other. In this case, there's no need to remove any of them. 

---
# Section 3: Data Pre-processing and Cleaning

Careful pre-processing is crucial for classical machine learning models, as they have limited tolerance for data and encoding issues compared to deep learning. Here we address:
- Missing/null values and type inconsistencies
- Outlier and skew handling
- Encoding categorical variables: binary, ordinal, nominal (one-hot), with justification for each feature
- Feature scaling (Standardization/Normalization), with method selection rationale
- Identifying important features for prediction


### 3.a) Handling Missing/NULL Values and Data Type Consistency
Datasets usually have missing values, and we cannot just leave them blank because most machine learning models can’t handle null values. So, I am fill (imputing) them with proper logic—for example, replacing with mean/median for numerical values, or mode for categorical values. The idea is to fill missing values in such a way that I don’t distort the dataset. Sometimes, missing data itself may carry meaning (like no internet service → missing online backup), so I also use domain knowledge when imputing.

- For this dataset, `TotalCharges` sometimes encodes missing values as blanks (''), so we must coerce and fix these. Other columns with `No internet service`/`No phone service` are valid categories, not missing values, so should be left as-is for categorical encoding.

**Logic I have used for imputation** For the imputation in TotalCharges, I forst checked if the tenure is 0 and the TotalCharges has a missing value, then in that case, I simply populated the TotalCharges with MonthlyCharges because the tenure is 0

In another case, I checked if the tenure is non zero, then I imputed the TotalCharges = tenure * MonthlyCharges

In [ ]:

df = df.replace(['', ' ', 'NA', 'na', 'NaN', 'null', 'NULL'], np.nan)


# Scan for missing values and datatypes
print('Missing values per column:')
print(df.isnull().sum())

df['TotalCharges'] = df['TotalCharges'].replace(' ', pd.NA)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

print(f"Missing TotalCharges before better imputation: {df['TotalCharges'].isna().sum()}")


# # Better imputation logic for missing values in TotalCharges

# # For 0 tenure customers, set TotalCharges to MonthlyCharges
zero_tenure_mask = (df['tenure'] == 0) & df['TotalCharges'].isna()
df.loc[zero_tenure_mask, 'TotalCharges'] = df.loc[zero_tenure_mask, 'MonthlyCharges']

# # For other missing values, estimate using tenure * MonthlyCharges  
missing_mask = df['TotalCharges'].isna()
df.loc[missing_mask, 'TotalCharges'] = df.loc[missing_mask, 'tenure'] * df.loc[missing_mask, 'MonthlyCharges']

print(f"Missing TotalCharges after better imputation: {df['TotalCharges'].isna().sum()}")


### 3.b) Outlier Handling

Outliers are values that are far away from the normal range of data. If I don’t handle them, they can pull the mean and standard deviation in a wrong direction, and the model might become biased. By visually inspecting (like boxplots, scatterplots), I can quickly spot unusual values and decide whether to keep, transform, or remove them. Outliers are not always bad, but I need to check them to make sure they don’t disturb the training process.

- For tenure and charges, I am using boxplots and statistical summaries to flag extreme values, then visually check distributions with log-transform or quantile capping if necessary (justified below).

- Result :- I have not found any outliers for this data set. So, there is no need to do this step for this data set.

We are lucky to not have outliers here BUT, For customer churn, outliers are likely legitimate (very high tenure/charges are loyal customers), so removal may lose valuable business insight. So In this case, not removing the outliers is a good choice as per design

In [ ]:
# Visual inspection for outliers
fig, axs = plt.subplots(1, 3, figsize=(18,4))
for i, col in enumerate(['tenure', 'MonthlyCharges', 'TotalCharges']):
    sns.boxplot(x=df[col], ax=axs[i], color='lightcoral')
    axs[i].set_title(f'Boxplot - {col}')
plt.tight_layout()
plt.show()

### 3.c) Skewness Detection

Skewness tells me whether the data is symmetric or not. If a column is highly skewed, then many algorithms (like logistic regression, linear regression) may not perform well because they assume data is more normally distributed.

In my case, the column ‘TotalCharges’ had many zero values, so I couldn’t apply log transformation (log of 0 is undefined). Instead, I used Yeo-Johnson transformation, which can handle zero and negative values as well. After transformation, the data becomes more balanced and closer to normal, which improves the model’s ability to learn patterns without being biased by extreme values.


In [ ]:
# Skewness check
skew_vals = df[['tenure','MonthlyCharges', 'TotalCharges']].skew()
print('Skewness of Numeric Features:')
print(skew_vals)

# Skewnesss of Tenure and MonthlyCharges are close to 0 -> So Transformation is not needed for them
# Transformation is needed for TotalCharges feature

I am dropping the customer Id from the dataset. Because, customer id is just an identifer and ther will be no logical relation with the output. So, removing that

In [ ]:
# Drop customerID
df_ml = df.drop('customerID', axis=1)

#Skewness of TotalCharges is  0.963316 - We need to reduce the skewness

from sklearn.preprocessing import PowerTransformer as pt

pt = pt(method="yeo-johnson")
df_ml['TotalCharges'] = pt.fit_transform(df_ml[["TotalCharges"]])

# Verify skewness after transformation
print('Skewness after yeo-johnson on TotalCharges:', df_ml['TotalCharges'].skew())

# Optional: Re-plot histogram for TotalCharges to visualize improvement
plt.figure(figsize=(6,4))
sns.histplot(df_ml['TotalCharges'], kde=True)
plt.title('Distribution of TotalCharges after log1p')
plt.show()


Result :- After I have applied the yeo-johnson Transformation on TotalCharges, the skewness value is  -0.14 -> Which is close to 0. So, now all our attributes are not skewed and ready fot further processing.

In [ ]:
# Recode redundant categories for internet-dependent features
internet_dependent = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
for col in internet_dependent:
    df_ml[col] = df_ml[col].replace('No internet service', 'No')

# Similarly for MultipleLines
df_ml['MultipleLines'] = df_ml['MultipleLines'].replace('No phone service', 'No')

### 3.d) Feature Engineering

Sometimes the raw dataset doesn’t directly capture useful information. Feature engineering is about creating new columns from the existing ones to improve the model. For example, combining tenure and Total charges might give a better signal about customer loyalty. By creating meaningful features, I can give the model more power to differentiate between classes. Basically, the quality of features often matters more than the complexity of the model. SO I have created various features here which are logically correct


In [ ]:
## Feature Engineering -> Creating new Features

# Create new predictive features
df_ml['AvgMonthlySpend'] = df_ml['TotalCharges'] / (df_ml['tenure'] + 1)
df_ml['IsMonthToMonth'] = (df_ml['Contract'] == 'Month-to-month').astype(int)
df_ml['IsElectronicCheck'] = (df_ml['PaymentMethod'] == 'Electronic check').astype(int)
df_ml['HasPartnerOrDependents'] = ((df_ml['Partner'] == 'Yes') | (df_ml['Dependents'] == 'Yes')).astype(int)
df_ml['HasPhoneAndInternet'] = ((df_ml['PhoneService'] == 'Yes') & (df_ml['InternetService'] != 'No')).astype(int)

# Count of additional services
service_cols = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
df_ml['AdditionalServices'] = sum([df_ml[col].map({'Yes': 1, 'No': 0}) for col in service_cols])


df_ml["is_long_contract"] = df_ml["Contract"].isin(["One year", "Two year"]).astype(int)
df_ml["fiber_streaming"] = (
    (df_ml["InternetService"] == "Fiber optic") & (df["StreamingTV"] == "Yes")
).astype(int)

### 3.e) Feature Scaling -> Z Standardization

**Why scale?** Classical ML models such as Logistic Regression (gradient-based), KNN (distance-based), and even tree ensembles (split-based, but still benefit) are sensitive to feature scale/variance. If numerical features vary on different scales, model could get biased or numerically unstable.

Standard Scaler transforms the data so that each feature has mean = 0 and standard deviation = 1. This is important because in many ML algorithms (like logistic regression, SVM, neural networks), features with larger values can dominate the smaller ones. By scaling, all features are put on the same scale, which helps the model converge faster and improves performance. Without scaling, optimization may become unstable and lead to poor results.

- **Which method to use?**
    - **StandardScaler (mean 0, std 1):** Best for features with approximately bell-shaped (normal) distributions, and for algorithms making use of means/stds or distance (KNN, LR).
    - **MinMaxScaler (0-1 scaling):** Used when features show non-Gaussian/skewed distributions. Keeps ranges between 0 and 1 (usually for algorithms relying on distance or bounded inputs).
    - **RobustScaler:** For numeric features with a lot of outliers or heavy skew, scales by median and IQR efficiently.

I am useing StandardScaler for 'tenure', 'MonthlyCharges', 'TotalCharges' as they are wide-range continuous and only mildly skewed. Here's the code for StandardScaler


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
num_cols = ['tenure','MonthlyCharges','TotalCharges', 'AvgMonthlySpend', 'AdditionalServices']
df_ml[num_cols] = scaler.fit_transform(df_ml[num_cols])

# Verify scaling
df_ml[num_cols].describe().T[['mean','std']]

### 3.f) Encoding Categorical Features

#### Justification:
For classical ML, all input features must be numerical. But how we transform categorical variables greatly affects model results, interpretability, and risks of data leakage.

**Encoding options include:**
- **Label Encoding:** Assigns unique integer to each category. Good only for ordinal features (if order is meaningful) to preserve rank. *Problem:* Can create artificial order when applied to nominal features (e.g., gender).
- **One-Hot Encoding:** Dummy variables for each category (except one as reference). Essential for truly nominal (unordered) features since it avoids false ordinality.
- **Binary Encoding:** For high-cardinality features (many unique categories), it encodes categories as binary digits, reducing dimensionality compared to one-hot, but less interpretable. Not needed here due to low cardinality.

#### Assignment of encoding method for each feature:
| Feature             | Encoder     | Justification                                                      |
|---------------------|-------------|--------------------------------------------------------------------|
| gender              | Binary/One-hot      | Two categories, no order: one-hot preferred for LR/Tree/Ensemble |
| SeniorCitizen       | Numeric (0/1) | Already numeric (0,1)                                             |
| Partner, Dependents | Binary/One-hot      | Two categories, no order, one-hot to avoid LR bias              |
| PhoneService        | Binary/One-hot      | Two categories, no order, one-hot                                |
| MultipleLines       | One-hot      | 3 values: Yes/No/No phone service, one-hot preserves all info     |
| InternetService     | One-hot      | 3 nominal values                                                  |
| OnlineSecurity, OnlineBackup, DeviceProtection, etc | One-hot | 3 levels including 'No internet service', one-hot most robust |
| Contract            | One-hot      | 3 unordered types                                                 |
| PaperlessBilling    | Binary/One-hot      | Two values, no order, one-hot to avoid coefficients bias in LR  |
| PaymentMethod       | One-hot      | 4 different methods, no order, one-hot for model reliability      |

We avoid Label Encoding for non-ordinal variables to prevent giving unintended priority/order to categories, especially since Logistic Regression and tree methods handle one-hots natively.

**Note:** I already dropped the customerID (non-predictive identifier) before modeling.

**Churn** will be mapped to 1/0 for modeling.

In [ ]:
print("Starting feature Encoding")

# Encode churn label
df_ml['Churn'] = df_ml['Churn'].map({'Yes': 1, 'No': 0})

print("Churn -> Output class mapped to 0 and 1")

# List categorical features (excluding target)
cat_feats = [col for col in df_ml.columns if df_ml[col].dtype == 'object' and col != 'Churn']

# Show unique values for categorical features
for col in cat_feats:
    print(f"{col}: {df_ml[col].unique()}")

# Apply one-hot encoding
df_ml = pd.get_dummies(df_ml, columns=cat_feats, drop_first=False)

print('Shape after encoding:', df_ml.shape)
df_ml.head()

### 3.g) Feature Importance Analysis for Engineering

**Why important:** In classical ML, removing uninformative or redundant features improves model generalization and speeds up training. 

I an checking feature importance to understand which variables are contributing the most to predictions. This is useful because it shows whether the model is depending on meaningful features (like tenure, contract type) or just on noise. By ranking features, I also get ideas for feature engineering and for reducing less important columns that don’t add much value.

In [ ]:
from sklearn.feature_selection import mutual_info_classif

X = df_ml.drop('Churn', axis=1)
y = df_ml['Churn']
mi_scores = mutual_info_classif(X, y, discrete_features='auto', random_state=42)

feat_importance = pd.Series(mi_scores, X.columns).sort_values(ascending=False)

# Top 15 features
feat_importance[:15].plot(kind='barh', figsize=(8,6))
plt.gca().invert_yaxis()
plt.title('Feature Importance (Mutual Information w/ Churn)')
plt.show()

# Display top features (table)
display(feat_importance.head(15).to_frame('MI Score'))

# Drop low-importance features (MI <= 0.01, keep important only)
selected_feats = feat_importance[feat_importance > 0.01].index.tolist()
if 'Churn' not in selected_feats:
    selected_feats.append('Churn')
df_ml = df_ml[selected_feats]


---
## Section 4: Model Building 

### Overview:
In this section, we develop and evaluate several classical machine learning models—Logistic Regression, Decision Tree, K-Nearest Neighbors, and an Ensemble (Random Forest)—to classify customer churn. We demonstrate the effect of different train/test splits and use cross-validation for hyperparameter tuning.


#### 4.a) Train Test Split

Splitting into train and test is very important. If I train and test on the same dataset, the model will just memorize the answers and show high accuracy, which is misleading. By splitting, I keep a part of the dataset hidden from training. Later, when I test, I can check whether the model can generalize to unseen data. This ensures that I am not overfitting and my performance results are genuine.

- The standard split for model validation is 80% train, 20% test. This provides enough data for training while holding out a portion for an unbiased evaluation. However, smaller or larger splits (such as 70/30, 90/10) can be tested for sensitivity.
- **80/20**: Widely accepted in industry; balances model learning and robust validation.
- **70/30**: More data for validation; may benefit overfit-prone models.
- **90/10**: More data for training; fewer for test—sometimes useful with smaller datasets or when model learning is challenging.


In [ ]:
from sklearn.model_selection import train_test_split
# Data for modeling
X = df_ml.drop('Churn', axis=1)
y = df_ml['Churn']

# 80/20 split (default)
X_train_80, X_test_20, y_train_80, y_test_20 = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"80/20 split - Train: {X_train_80.shape}, Test: {X_test_20.shape}")

# 70/30 split
X_train_70, X_test_30, y_train_70, y_test_30 = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
print(f"70/30 split - Train: {X_train_70.shape}, Test: {X_test_30.shape}")

# 90/10 split
X_train_90, X_test_10, y_train_90, y_test_10 = train_test_split(X, y, test_size=0.10, random_state=42, stratify=y)
print(f"90/10 split - Train: {X_train_90.shape}, Test: {X_test_10.shape}")

#### 4.b) Addressing the Class Imbalance problem using SMOTE - Synthetic Minority Oversampling Technique

In my dataset, the classes are imbalanced (very few churn cases compared to non-churn). If I don’t fix this, the model will always predict the majority class and still show high accuracy, but it will fail to capture churners.

So, I applied SMOTE (Synthetic Minority Oversampling Technique) to generate new synthetic examples of the minority class. After that, I also used SMOTEENN (combination of SMOTE + Edited Nearest Neighbors) which balances the data better by removing overlapping/noisy samples. This way, the model learns to give attention to both classes.


In [ ]:
print('Churn distribution in full set:', y.mean().round(3))
print('Churn in 80% train:', y_train_80.mean().round(3))
print('Churn in 20% test:', y_test_20.mean().round(3))
# Check other splits
print('Churn in 70% train:', y_train_70.mean().round(3))
print('Churn in 30% test:', y_test_30.mean().round(3))
print('Churn in 90% train:', y_train_90.mean().round(3))
print('Churn in 10% test:', y_test_10.mean().round(3))

# Applying SMOTE to address class imbalance on training sets only

from imblearn.combine import SMOTEENN
smote = SMOTEENN(random_state=42)

X_train_80_bal, y_train_80_bal = smote.fit_resample(X_train_80, y_train_80)
X_train_70_bal, y_train_70_bal = smote.fit_resample(X_train_70, y_train_70)
X_train_90_bal, y_train_90_bal = smote.fit_resample(X_train_90, y_train_90)


# Verify balance
print('Balanced 80/20 train:', y_train_80_bal.mean().round(3))
print('Balanced 70/30 train:', y_train_70_bal.mean().round(3))
print('Balanced 90/10 train:', y_train_90_bal.mean().round(3))

## _Important NOTE_
I first did the train-test split and then applied SMOTE only on the training set. After researching on this, I have found that This is the correct and recommended practice because applying SMOTE before splitting would allow synthetic data to leak into the test set, which gives unrealistically high results.

In fact, when I tried the reverse way i.e., Applying SMOTE first and then splitting the data, I was getting accuracy and F1 scores around 98%, which is clearly misleading. So, I am consciously following the best practice here to make sure my evaluation is fair and reliable.

#### 4.c) Logistic Regression + Hyperparameter Tuning

I tuned Logistic Regression using GridSearchCV. The parameters I experimented with are:

Penalty (l1, l2) → decides how regularization is applied (to prevent overfitting).

C (0.1, 0.5, 1, 2, 3) → inverse of regularization strength. Smaller C means stronger regularization and higher penalty in case of oversampling.

class_weight (balanced / none) → helps handle imbalance.

GridSearchCV helps me test different combinations automatically and pick the best set. Logistic regression is simple, interpretable, and gives me a baseline model.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

logreg = LogisticRegression(solver='liblinear', max_iter=1000, class_weight='balanced')
# Tune penalty and regularization strength
param_grid_lr = {
    'penalty': ['l1', 'l2'], 
    'C': [0.1, 0.5, 1, 2, 3],
    'class_weight': ['balanced', None]
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Grid search for optimal hyperparameters on 80/20 split
gs_lr = GridSearchCV(logreg, param_grid_lr, cv=cv, scoring='f1', n_jobs=-1)
gs_lr.fit(X_train_80_bal, y_train_80_bal)
print(f"Best LR params: {gs_lr.best_params_}; best CV ROC AUC: {gs_lr.best_score_:.3f}")

# Predict on test
y_pred_lr = gs_lr.predict(X_test_20)
y_proba_lr = gs_lr.predict_proba(X_test_20)[:,1]

print("Accuracy:", accuracy_score(y_test_20, y_pred_lr))
print("Precision:", precision_score(y_test_20, y_pred_lr))
print("Recall:", recall_score(y_test_20, y_pred_lr))
print("F1 Score:", f1_score(y_test_20, y_pred_lr))
print("ROC AUC:", roc_auc_score(y_test_20, y_proba_lr))


#### 4.d) Decision Tree Classifier + Hyperparameter Tuning

- It is very much Flexible, interpretable for business rules by using the Decision Trees
- Hyperparameters control overfitting (max_depth, min_samples_split).

For Decision Trees, I tuned the following:

max_depth (10, 15, 20, 30) → controls how deep the tree can go. Prevents overfitting.

min_samples_split (2, 5, 10, 20) → minimum samples required to split a node.

min_samples_leaf (1, 2, 5) → ensures leaves have enough data.

class_weight (balanced / none) → to deal with imbalance.

##### Justification: Decision Tree Tuning
- `max_depth` prevents overly complex trees (risk of overfit).
- `min_samples_split` prevents splits on tiny samples that overfit noise.
- Values are chosen to allow both shallow and deep trees for comparison.

Decision Trees are good because they can capture non-linear relationships, but they can overfit, so tuning is very important.


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42, class_weight='balanced')
param_grid_dt = {
    'max_depth': [10, 15, 20, 30],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5],
    'class_weight': ['balanced', None]
}
gs_dt = GridSearchCV(dt, param_grid_dt, cv=cv, scoring='f1', n_jobs=-1)
gs_dt.fit(X_train_80_bal, y_train_80_bal)
print(f"Best Decision Tree params: {gs_dt.best_params_}; best CV ROC AUC: {gs_dt.best_score_:.3f}")
y_pred_dt = gs_dt.predict(X_test_20)
y_proba_dt = gs_dt.predict_proba(X_test_20)[:,1]

print("Accuracy:", accuracy_score(y_test_20, y_pred_dt))
print("Precision:", precision_score(y_test_20, y_pred_dt))
print("Recall:", recall_score(y_test_20, y_pred_dt))
print("F1 Score:", f1_score(y_test_20, y_pred_dt))
print("ROC AUC:", roc_auc_score(y_test_20, y_proba_dt))

#### 4.e) K-Nearest Neighbors (KNN) + Hyperparameter Tuning

For KNN, I tuned:

n_neighbors (1, 3, 5) → number of nearest neighbors considered.

weights (uniform, distance) → whether all neighbors are equal or closer ones are weighted more.

metric (euclidean, manhattan) → distance calculation method.

KNN is simple but sensitive to scaling and outliers, so I used StandardScaler before applying it.

##### Justification: KNN Tuning
- `n_neighbors`: balances bias-variance (small = highly flexible, large = smoother)
- `weights`: uniform for classic, distance for soft-voting
- `metric`: explores different distance measures (Minkowski covers Euclidean and Manhattan)
- Scaling/standardization is especially crucial, handled above.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
param_grid_knn = {
    'n_neighbors': [1, 3, 5],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}
gs_knn = GridSearchCV(knn, param_grid_knn, cv=cv, scoring='f1', n_jobs=-1)
gs_knn.fit(X_train_80_bal, y_train_80_bal)
print(f"Best KNN params: {gs_knn.best_params_}; best CV ROC AUC: {gs_knn.best_score_:.3f}")
y_pred_knn = gs_knn.predict(X_test_20)
y_proba_knn = gs_knn.predict_proba(X_test_20)[:,1]

print("Accuracy:", accuracy_score(y_test_20, y_pred_knn))
print("Precision:", precision_score(y_test_20, y_pred_knn))
print("Recall:", recall_score(y_test_20, y_pred_knn))
print("F1 Score:", f1_score(y_test_20, y_pred_knn))
print("ROC AUC:", roc_auc_score(y_test_20, y_proba_knn))

#### 4.e) Random Forest Classifier + Hyperparameter Tuning

- Robust, handles both categorical and numerical, reduces variance.
- Primary tuning: number of trees (n_estimators), max_depth.

For Random Forest, I tuned:

n_estimators (100, 200, 300) → number of trees in the forest.

max_depth (10, 15, None) → limit of tree depth.

min_samples_split (2, 5, 10) → avoids too fine splits.

class_weight (balanced / none).

Random Forests are powerful because they average results across many trees, reducing overfitting and improving stability.

##### Justification: Random Forest Tuning
- `n_estimators`: More trees = greater stability, but with diminishing returns after around 100-200.
- `max_depth`: Controls tree size/complexity. None = unpruned.
- `min_samples_split`: Prevents overfitting to small branches.
- RF is robust to uninformative features due to built-in selection, so we use the whole set post-encoding.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42, class_weight='balanced')
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 15, None],
    'min_samples_split': [2, 5, 10],
    'class_weight': ['balanced', None]
}
gs_rf = GridSearchCV(rf, param_grid_rf, cv=cv, scoring='f1', n_jobs=-1)
gs_rf.fit(X_train_80_bal, y_train_80_bal)
print(f"Best RF params: {gs_rf.best_params_}; best CV ROC AUC: {gs_rf.best_score_:.3f}")
y_pred_rf = gs_rf.predict(X_test_20)
y_proba_rf = gs_rf.predict_proba(X_test_20)[:,1]

print("Accuracy:", accuracy_score(y_test_20, y_pred_rf))
print("Precision:", precision_score(y_test_20, y_pred_rf))
print("Recall:", recall_score(y_test_20, y_pred_rf))
print("F1 Score:", f1_score(y_test_20, y_pred_rf))
print("ROC AUC:", roc_auc_score(y_test_20, y_proba_rf))

#### 4.f) Ensemble Learning – Why I Used It

I used an ensemble of: XGBoost Classifier (xgb_clf) + LightGBM Classifier (lgbm_clf) + Random Forest Classifier (rf_clf)

Ensemble models combine the strengths of multiple algorithms. For example, XGBoost and LightGBM are boosting algorithms that focus on misclassified samples, while Random Forest is a bagging method that reduces variance. Using them together gave me stronger results compared to individual models.

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


# Define base models
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
lgbm_clf = LGBMClassifier(random_state=42)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Create voting classifier (hard voting = majority vote)
voting_clf = VotingClassifier(
    estimators=[
        ('xgb', xgb_clf),
        ('lgbm', lgbm_clf),
        ('rf', rf_clf)
    ],
    voting='soft'  # Use 'soft' if you want to average predicted probabilities
)

# Fit and predict
voting_clf.fit(X_train_80_bal, y_train_80_bal)
y_pred_ensemble = voting_clf.predict(X_test_20)
y_proba_ensemble = voting_clf.predict_proba(X_test_20)[:,1]


# Evaluate
print("Voting Classifier Accuracy:", accuracy_score(y_test_20, y_pred_ensemble))
print("Voting Classifier Precision:", precision_score(y_test_20, y_pred_ensemble))
print("Voting Classifier Recall:", recall_score(y_test_20, y_pred_ensemble))
print("Voting Classifier F1 Score:", f1_score(y_test_20, y_pred_ensemble))
print("Voting Classifier ROC AUC:", roc_auc_score(y_test_20, y_proba_ensemble))


Hyperparameter Tuning Results + Precision vs Recall

Across all models, I observed that:

Recall was high (~82%) → meaning the model is catching most of the churn cases.

Precision was low (~50%) → meaning many of the predicted churns were actually false alarms.

This trade-off happened because of the dataset itself—class imbalance + overlapping feature values made it harder to get high precision. Since recall is more important in churn prediction (we don’t want to miss churners), I accepted this trade-off, but noted that precision kept the F1 score around 60%.

---
# Section 5: Performance Evaluation & Selection of the Best Model

Now we will evaluate all models using classical metrics for binary classification—**Accuracy, Precision, Recall, F1-Score**, and **ROC-AUC**—on the held-out test set.

### Why these metrics?
- **Accuracy:** Simple overall correctness; less reliable for imbalanced classes.
- **Precision/Recall/F1:** Handle class imbalance, reflect business concern about false approvals (precision) and missed at-risk customers (recall).
- **ROC-AUC:** Comprehensive—measures model's ability to rank churners above non-churners regardless of threshold; best for business action ranking.
- **Justification:** Using all together ensures robust selection; ROC-AUC is often considered most reliable for churn, F1 balances miss and cost.

In [ ]:
# Calculate all metrics for test set (80/20)
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix

def eval_metrics(y_true, y_pred, y_proba, name):
    return {
        'Model': name,
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1': f1_score(y_true, y_pred),
        'ROC_AUC': roc_auc_score(y_true, y_proba)
    }

results = []
results.append(eval_metrics(y_test_20, y_pred_lr, y_proba_lr, 'Logistic Regression'))
results.append(eval_metrics(y_test_20, y_pred_dt, y_proba_dt, 'Decision Tree'))
results.append(eval_metrics(y_test_20, y_pred_knn, y_proba_knn, 'KNN'))
results.append(eval_metrics(y_test_20, y_pred_rf, y_proba_rf, 'Random Forest'))
results.append(eval_metrics(y_test_20, y_pred_ensemble, y_proba_ensemble, 'Voting Classifier[Ensemble]'))


perf_df = pd.DataFrame(results).set_index('Model')
display(perf_df)

## Visualization: Comparison of Evaluation Metrics Across Models

**Justification:**
A bar/chart plot allows for immediate visual benchmarking across all performance metrics for each model. This is the standard for model comparison and helps communicate results to technical and non-technical stakeholders.

In [ ]:
# Plot performance metrics for all models
plt.figure(figsize=(10,6))
perf_df[['Accuracy','Precision','Recall','F1','ROC_AUC']].plot(kind='bar', ax=plt.gca(), rot=15)
plt.title('Comparison of Model Performance Metrics')
plt.ylabel('Score')
plt.ylim(0, 1)
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

## ROC Curves for All Models

Visualizing ROC-AUC curves side by side gives a nuanced view of true positive/false positive trade-offs and allows comparison at every threshold.

In [ ]:
plt.figure(figsize=(8,6))
models_pred = [
    ('Logistic Regression', y_proba_lr),
    ('Decision Tree', y_proba_dt),
    ('KNN', y_proba_knn),
    ('Random Forest', y_proba_rf),
    ('Voting Classifier[Ensemble]', y_proba_ensemble)
]
for name, yproba in models_pred:
    fpr, tpr, _ = roc_curve(y_test_20, yproba)
    plt.plot(fpr, tpr, label=f'{name} (AUC: {roc_auc_score(y_test_20, yproba):.2f})')
plt.plot([0,1],[0,1],'k--', lw=0.8)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend()
plt.tight_layout()
plt.show()

### Which Model is Best? (and Why)

**Best Model Selection (with Justification)**

- Model selection should be based primarily on ROC-AUC and F1-score, as they provide robust indicators of balance between generalization and actionable predictive value on imbalanced classification problems like churn.
- From the metrics and ROC curve above, the model with the highest AUC and F1, while maintaining strong precision/recall, should be chosen. In customer churn, ROC-AUC >0.84 is considered highly effective.

**Justification:**
- **Random Forest** usually outperforms others here, due to its ability to model nonlinear relationships and handle interactions between features robustly and without the need for explicit transformation or pruning.
- Logistic Regression offers interpretability, but is outperformed on nonlinear interaction data.
- Decision Tree is interpretable but often overfits or underfits compared to ensembles, as shown by lower test AUC/F1.
- KNN, while useful for simple separation, performs worse on mixed/complex categorical+numeric features and high dimensionality after encoding.

**Conclusion:**
> _The best classical machine learning model for this Telco Customer Churn prediction task is the **Random Forest classifier**, as evidenced by the highest ROC-AUC, F1, and balanced metrics in both visual and tabular comparisons above._

Random Forest should be deployed for customer churn scoring but logistic regression coefficients may still be referenced for business explainability of individual attributes.